### 모델 예측 해 보기

####  1. 준비: 필요한 코드 가져오기

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tensorflow.keras.models import load_model

In [2]:
import tensorflow as tf

In [3]:
# 1. 사전 로딩된 LSTM 모델
model = load_model('./new_model/29-0.7179.keras')

3. 사용자 입력 한 데이터

In [4]:
# 만약 사용자가 날짜, 발주수량을 3일치 입력한다고 가정한다.

orders = [
    {"date" :"2025-04-10", "qty" : 200 },
    {"date" :"2025-04-09", "qty" : 500 },
    {"date" :"2025-04-11", "qty" : 100 }
    ]

# 날짜는 반드시 'YYYY-MM-DD' 문자열로 입력해야 한다.

4. 데이터 변환 함수

In [5]:
pd.DataFrame(orders)

,date,qty
0,2025-04-10,200
1,2025-04-09,500
2,2025-04-11,100


In [6]:
df = pd.DataFrame(orders)
df['date'] = pd.to_datetime(df['date'])
df['date']

0   2025-04-10
1   2025-04-09
2   2025-04-11
Name: date, dtype: datetime64[ns]

In [7]:
df = df.sort_values(by='date')
df

,date,qty
1,2025-04-09,500
0,2025-04-10,200
2,2025-04-11,100


In [8]:
#가장 마지막 날짜 구하기
latest_date = df['date'].max()
latest_date

Timestamp('2025-04-11 00:00:00')

In [ ]:
lookback_days = 3   #LSTM이 과거 3일 데이터를 입력으로 사용(즉, 타임스텝 3)
range(lookback_days - 1, -1, -1)   #2에서 -1까지(불포함) -1씩 감소 => 결과 [2,1,0]

#최근 날짜에서 2일전->1일전->당일 순서로 반복

range(2, -1, -1)

In [11]:
# timedelta(days=i)로 날짜 차이 계산

print("기준일(당일)", latest_date - timedelta(0))
print("하루전", latest_date - timedelta(days=1))
print("2일전", latest_date-timedelta(2))

기준일(당일) 2025-04-11 00:00:00
하루전 2025-04-10 00:00:00
2일전 2025-04-09 00:00:00


In [12]:
# 이 코드는 LSTM 모델에서 예측을 위한 시계열 입력을 구성하기 위해 "3일치 날짜를 최근 날짜 기준으로 거꾸로 생성"
date_range = [latest_date - timedelta(days=i) for i in range(lookback_days - 1, -1, -1) ]
date_range

[Timestamp('2025-04-09 00:00:00'),
 Timestamp('2025-04-10 00:00:00'),
 Timestamp('2025-04-11 00:00:00')]

In [13]:
df[df['date'] == date_range[0]]

,date,qty
1,2025-04-09,500


In [ ]:
# 만약 연속적인 날짜(3일)이 필요한데, 누락된 데이터가 있다면, 연습
filled_empty = []
df_empty = pd.DataFrame([
    {'date': '2025-4-9', 'qty': 100},
    {'date': '2025-4-11', 'qty': 300}
])

# 날짜 타입 변환!
df_empty['date'] = pd.to_datetime(df_empty['date'])

#기준일에서 -1, -2한 날짜, 연속된 날짜 3개
date_range_empty = [datetime(2025, 4, 9), datetime(2025, 4, 10), datetime(2025, 4, 11)]  

for date in date_range_empty :
    row = df_empty[df_empty['date'] == date]
    if row.empty :
        filled_empty.append({"date" : date, "qty" : 0})
    else:
        filled_empty.append({"date" : date, "qty" : int(row['qty'].values[0])})
pd.DataFrame(filled_empty)

,date,qty
0,2025-04-09,100
1,2025-04-10,0
2,2025-04-11,300


In [15]:
# LSTM 모델에 넣기 위한 날짜별 발주 수량 데이터를 정리
# date_range에 해당하는 날짜 3일치에 대해 입력 데이터(df)에 해당 날짜가 없으면 
# → 수량 0으로 채우고, 있으면 → 해당 날짜의 수량을 넣는다
# 날짜가 연속적이여야 하고, 누락된 날짜는 반드시 채워줘야 함(0 또는 다른 값으로)

filled = []
for date in date_range :
    row = df[df['date'] == date]
    if row.empty :
        filled.append({"date" : date, "qty" : 0})
    else:
        filled.append({"date" : date, "qty" : int(row['qty'].values[0])})
pd.DataFrame(filled)

,date,qty
0,2025-04-09,500
1,2025-04-10,200
2,2025-04-11,100


In [16]:
filled_df = pd.DataFrame(filled)
filled_df

,date,qty
0,2025-04-09,500
1,2025-04-10,200
2,2025-04-11,100


In [17]:
qtys = filled_df['qty'].values
qtys

array([500, 200, 100], dtype=int64)

In [ ]:
# LSTM에 입력할 시계열 데이터를 만든다
# 3일치 발주 수량 데이터(qtys)를 다양한 수치로 변환해서 8개의 피처(feature)를 추출

# LSTM 입력 형태: (samples, time_steps=3, features=8)
# → 각 날짜마다 8가지 피처(숫자)를 만들어야 하므로 이 반복문에서 8개의 값 추출

In [18]:
len(qtys)

3

In [21]:
#현재 날짜의 수량
qtys[2]

100

In [30]:
qtys[:3 + 1]  #qtys[:4]  처음부터 3번까지 가져온다.

array([500, 200, 100], dtype=int64)

In [31]:
for i in range(len(qtys)):
    mean_qty = np.mean(qtys[:i+1])  # 지금까지 평균
mean_qty

266.6666666666667

In [ ]:
# 평균 수량 (누적 평균)
# i(3)이 0보다 크면 qtys배열을 가져와 평균값을 내고, 그렇지 않으면 qtys[i]값을 가져온다
mean_qty = np.mean(qtys[:3+1]) if 3 > 0 else qtys[3]
mean_qty

# A if 조건 else B
# 조건이 True이면 A를 실행, 조건이 False이면 B를 실행

266.6666666666667

In [ ]:
# 증감량 (직전일 대비 차이)

diff = qtys[i] - qtys[i - 1] if i>0 else 0

# 이전날과의 차이, 첫날은 비교 대상 없으므로 0

In [ ]:
#누적 수량
cumulative = np.sum(qtys[:i + 1])

# i=2이면 qty[0] + qty[1] + qty[2]
# 현재까지 누적 발주량

In [ ]:
# 현재 수량 / 평균

qtys[0] / (mean_qty + 1e-5)

# 어떤 수를 0으로 나누면 ZeroDivisionError 발생
# 1e-5는 "0으로 나누는 오류(Division by Zero)"를 피하기 위한 안전장치

1.8749999999999998

In [36]:
# 2. 최종 피처 벡터 구성 (8개)
features = []
for i in range(len(qtys)):
    qty = qtys[i]
    mean_qty = np.mean(qtys[:i + 1]) if i > 0 else qty
    diff = qtys[i] - qtys[i - 1] if i > 0 else 0
    cumulative = np.sum(qtys[:i + 1])
    features.append([
        qty,            # 1. 현재 수량
        mean_qty,       # 2. 평균 수량
        diff,           # 3. 증감량
        cumulative,     # 4. 누적 수량
        1 if qty > 0 else 0,   # 5. 발주 여부 (Binary)
        i / lookback_days,     # 6. 상대 시간 위치 (0.0, 0.33, 0.66)  #lookback_days=3
        np.log1p(qty),         # 7. 로그 스케일 수량 (log(1+qty))
        qty / (mean_qty + 1e-5)    # 8. 현재 수량 / 평균 (비율)
    ])
features

[[500, 500, 0, 500, 1, 0.0, 6.2166061010848646, 0.9999999800000005],
 [200,
  350.0,
  -300,
  700,
  1,
  0.3333333333333333,
  5.303304908059076,
  0.5714285551020413],
 [100,
  266.6666666666667,
  -100,
  800,
  1,
  0.6666666666666666,
  4.61512051684126,
  0.37499998593750056]]

In [ ]:
np.array(features).reshape(1, 3, 8)

# reshape(1, 3, 8)  : 1개의 샘플, 3일치 데이터, 8개의 피처로 모델에 입력하기

array([[[ 5.00000000e+02,  5.00000000e+02,  0.00000000e+00,
          5.00000000e+02,  1.00000000e+00,  0.00000000e+00,
          6.21660610e+00,  9.99999980e-01],
        [ 2.00000000e+02,  3.50000000e+02, -3.00000000e+02,
          7.00000000e+02,  1.00000000e+00,  3.33333333e-01,
          5.30330491e+00,  5.71428555e-01],
        [ 1.00000000e+02,  2.66666667e+02, -1.00000000e+02,
          8.00000000e+02,  1.00000000e+00,  6.66666667e-01,
          4.61512052e+00,  3.74999986e-01]]])

In [38]:
# 지금까지 연습한 것을 메서드로 만듬
def convert_user_input_to_model_input(orders):
    df = pd.DataFrame(orders)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(by='date')

    latest_date = df['date'].max()
    lookback_days = 3
    date_range = [latest_date - timedelta(days=i) for i in range(lookback_days - 1, -1, -1)]

    filled = []
    for date in date_range:
        row = df[df['date'] == date]
        if row.empty:
            filled.append({"date": date, "qty": 0})
        else:
            filled.append({"date": date, "qty": int(row['qty'].values[0])})

    filled_df = pd.DataFrame(filled)
    qtys = filled_df['qty'].values

    features = []
    for i in range(len(qtys)):
        qty = qtys[i]
        mean_qty = np.mean(qtys[:i + 1]) if i > 0 else qty
        diff = qtys[i] - qtys[i - 1] if i > 0 else 0
        cumulative = np.sum(qtys[:i + 1])
        features.append([
            qty,
            mean_qty,
            diff,
            cumulative,
            1 if qty > 0 else 0,
            i / lookback_days,
            np.log1p(qty),
            qty / (mean_qty + 1e-5)
        ])

    return np.array(features).reshape(1, 3, 8)


5. 변환 + 예측 실행

In [40]:
model_input = convert_user_input_to_model_input(orders)
print("LSTM 입력 형태:", model_input.shape)

LSTM 입력 형태: (1, 3, 8)


In [45]:
# 예측
y_pred = model.predict(model_input).flatten()[0]
print(f"📦 예측된 3일 후 발주 수량: {y_pred:.2f} 개")

# 이모지 :  윈도우 키 + .
# f는 f-string (formatted string literal)의 표시
# 문자열 포맷팅 방식으로, 문자열 안에 변수나 표현식을 {} 중괄호로 직접 넣을 수 있다

1/1 [==============================] - 0s 44ms/step
📦 예측된 3일 후 발주 수량: 298.03 개
